In [1]:
%pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable
                                              0.0/9.7 MB ? eta -:--:--
     --                                       0.7/9.7 MB 13.9 MB/s eta 0:00:01
     -----                                    1.2/9.7 MB 13.2 MB/s eta 0:00:01
     -------                                  1.8/9.7 MB 12.6 MB/s eta 0:00:01
     ----------                               2.6/9.7 MB 13.9 MB/s eta 0:00:01
     -----------                              2.9/9.7 MB 14.1 MB/s eta 0:00:01
     --------------                           3.5/9.7 MB 12.5 MB/s eta 0:00:01
     ---------------                          3.8/9.7 MB 11.5 MB/s eta 0:00:01
     -------------------                      4.7/9.7 MB 12.5 MB/s eta 0:00:01
     --------------------                     4.9/9.7 MB 11.6 MB/s eta 0:00:01
     -----------------------                  5.7/9.7 MB 12.2 MB/s eta 0:00:01
     --------------------------               6.3/9.7 MB 12.3


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Facility Location

The facility location problem is a classic optimization challenge in operations research and logistics. It involves finding the best locations for facilities like warehouses, factories, or service centers to meet demand while minimizing costs. Factors such as transportation costs, customer locations, and facility capacities are considered in the decision-making process. The goal is to optimize the distribution network and ensure efficient service to customers or clients. Various mathematical and algorithmic approaches are used to solve this problem and make well-informed location decisions.

## Problem Description

A large supermarket chain in the CANADA needs to build warehouses for a set of supermarkets. The 50 locations of the supermarkets have been identified, but the locations of the warehouses have yet to be determined.

20 good candidate locations for the warehouses have been identified, but decisions must be made regarding how many warehouses to open and at which candidate locations to build them.

Opening many warehouses would be advantageous as this would reduce the average distance a truck has to drive from the warehouse to the supermarket, and hence reduce the delivery cost. However, opening a warehouse has a fixed cost associated with it. Here, in this problem cost per mile is two million GBP.


In this example, our goal is to find the optimal tradeoff between delivery costs and the costs of building new facilities.

## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex business problem. 

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

### Importing Libraries

In [31]:
import pandas as pd
from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import GRB


### Reading Datasets

In [32]:
df_prod=pd.read_csv(r"C:\Users\DELL\Downloads\Production_FLP.csv")
df_demand=pd.read_csv(r"C:\Users\DELL\Downloads\Demand_FLP.csv")
print(df_prod.head())
print(df_demand.tail())

   Serial No. Warehouse_Location  Production_Unit  Fixed_cost(million)  \
0           1          Ladysmith            89711                    3   
1           2            Langely            26033                    1   
2           3           Langford            68890                    9   
3           4            Langley            45304                    4   
4           5        Maple Ridge            52135                    4   

   Longitude  Latitude  
0    -123.80     48.98  
1    -122.66     49.11  
2    -123.50     48.46  
3    -122.66     49.22  
4    -120.76     50.12  
    Serial_No. Supermarket_Location  Demand_unit  Longitude  Latitude
45          46               Fernie        37058    -119.49     49.86
46          47         Fort St John        66488    -118.22     51.01
47          48              Gibsons        76307    -123.18     49.15
48          49             Kamloops        75821    -123.38     48.46
49          50              Kelowna        57689    -11

### Parameters

In [33]:
facilities= list(zip(df_prod['Longitude'], df_prod['Latitude']))
customers= list(zip(df_demand['Longitude'], df_demand['Latitude']))
setup_cost= list(df_prod['Fixed_cost(million)'])
cost_per_mile=2
print(setup_cost) #million

[3, 1, 9, 4, 4, 2, 1, 5, 10, 3, 9, 5, 6, 7, 6, 1, 5, 2, 2, 6]


### Preprocessing

In [34]:
# This function determines the  distance between a facility and customer sites.

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

# Compute key parameters of MIP model formulation
num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

# Compute shipping costs

shipping_cost = {(c,f): cost_per_mile*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}
print(shipping_cost) #transportation_cost

{(0, 0): 20.14747626875387, (0, 1): 17.870030777813454, (0, 2): 19.832922124588713, (0, 3): 17.81812560287979, (0, 4): 13.733957914599877, (0, 5): 17.200988343696995, (0, 6): 20.525545059754204, (0, 7): 7.471813702174339, (0, 8): 18.305070335838654, (0, 9): 18.597817076205484, (0, 10): 20.985575998766397, (0, 11): 11.758248168838778, (0, 12): 17.871944494094656, (0, 13): 21.967248348393593, (0, 14): 18.0337683250063, (0, 15): 18.122439129432887, (0, 16): 21.25331033039324, (0, 17): 18.36154677580297, (0, 18): 33.2211077479364, (0, 19): 18.18532375296081, (1, 0): 17.037945885581404, (1, 1): 14.77200054156512, (1, 2): 16.767027166435927, (1, 3): 14.711859161914248, (1, 4): 10.59428147634376, (1, 5): 14.108862463005298, (1, 6): 17.398160822339822, (1, 7): 4.794371700233525, (1, 8): 15.194367377419837, (1, 9): 15.478630430370778, (1, 10): 17.851890656174216, (1, 11): 8.72935278242324, (1, 12): 14.764403137275828, (1, 13): 18.831516136519674, (1, 14): 14.922533296997546, (1, 15): 15.0090106

### Model Deployment

In [35]:
# MIP  model formulation

m = gp.Model('facility_location')

select = m.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
assign = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='Assign')

m.addConstrs((assign[(c,f)] <= select[f] for c,f in cartesian_prod), name='Setup2ship')
m.addConstrs((gp.quicksum(assign[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

m.setObjective(select.prod(setup_cost)+assign.prod(shipping_cost), GRB.MINIMIZE)

m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1050 rows, 1020 columns and 3000 nonzeros
Model fingerprint: 0xe4d1b91e
Variable types: 1000 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-02, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 810.7645571
Presolve time: 0.00s
Presolved: 1050 rows, 1020 columns, 3000 nonzeros
Variable types: 1000 continuous, 20 integer (20 binary)

Root relaxation: objective 3.123268e+02, 58 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     312.3267597  312.32676

## Analysis


The result of the optimization model shows that the minimum total cost value is 3.12 million GBP. 
Let's see the solution that achieves that optimal result.

### Warehouse Build Plan

This plan determines at which site locations to build a warehouse.

In [39]:
# display optimal values of decision variables

for facility in select.keys():
    if (abs(select[facility].x) > 1e-6):
        print(f"\n Build a warehouse at location {df_prod.iloc[facility + 1]['Warehouse_Location']}.")


 Build a warehouse at location Mission.

 Build a warehouse at location Nanaimo.

 Build a warehouse at location Nelson.

 Build a warehouse at location Penticton.

 Build a warehouse at location Port Moody.

 Build a warehouse at location  Prince George.


### Shipment Plan 

This plan determines the percentage of shipments to be sent from each facility built to each customer.

In [42]:
# Shipments from facilities to customers.

for customer, facility in assign.keys():
    if (abs(assign[customer, facility].x) > 1e-6):
        print(f"\n Supermarket {df_demand.iloc[customer]['Supermarket_Location']} receives {round(100*assign[customer, facility].x, 2)} % of its demand  from Warehouse {df_prod.iloc[facility + 1]['Warehouse_Location']} .")



 Supermarket  Airdrie receives 100.0 % of its demand  from Warehouse Nelson .

 Supermarket  Banff receives 100.0 % of its demand  from Warehouse Nelson .

 Supermarket  Beaumont receives 100.0 % of its demand  from Warehouse Nelson .

 Supermarket Brooks receives 100.0 % of its demand  from Warehouse Nelson .

 Supermarket  Calgary receives 100.0 % of its demand  from Warehouse Nelson .

 Supermarket  Camrose receives 100.0 % of its demand  from Warehouse Nelson .

 Supermarket Canmore receives 100.0 % of its demand  from Warehouse Nelson .

 Supermarket Chestermere receives 100.0 % of its demand  from Warehouse Nelson .

 Supermarket  Cochrane receives 100.0 % of its demand  from Warehouse Nelson .

 Supermarket  Edmonton receives 100.0 % of its demand  from Warehouse Nelson .

 Supermarket  Fort McMurray receives 100.0 % of its demand  from Warehouse  Prince George .

 Supermarket Fort Saskatchewan receives 100.0 % of its demand  from Warehouse Nelson .

 Supermarket Grand Prairie 